# FastAPI Text Classification and Sentiment Analysis

This project sets up a FastAPI application that performs text classification using the Hugging Face `transformers` library. The application uses a sentiment analysis model, but this can be customized for other NLP tasks. It allows requests from any origin using CORS middleware and runs in a Jupyter Notebook environment with `nest_asyncio` to avoid event loop issues.

### Required Libraries

To run the code, you need to install the following libraries:

```bash
!pip install fastapi uvicorn transformers torch nest_asyncio


In [3]:
# Required libraries
#!pip install fastapi uvicorn transformers torch nest_asyncio

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from transformers import pipeline
import nest_asyncio
import uvicorn
from threading import Thread

# Apply nest_asyncio to fix event loop issues in Jupyter
nest_asyncio.apply()

# Load the model for text classification
classifier = pipeline('sentiment-analysis')  # You can change the task if needed

# Create FastAPI app
app = FastAPI()

# Add CORS middleware to allow access from the browser
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins
    allow_methods=["*"],  # Allow all methods
    allow_headers=["*"]   # Allow all headers
)

# Define a route for text classification
@app.get("/classify/{text}")
async def classify(text: str):
    result = classifier(text)
    return {"label": result[0]['label'], "score": result[0]['score']}

# Function to run FastAPI in background
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start FastAPI in a separate thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


INFO:     Started server process [932]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


# Text Classification API with FastAPI and Hugging Face's DistilBERT

This project implements a **text classification** API using **FastAPI** and Hugging Face's pre-trained **DistilBERT** model. The API allows you to classify input text into categories such as "POSITIVE", "NEGATIVE", or "NEUTRAL" based on the model's predictions. It is suitable for use cases like sentiment analysis or simple text classification.

## Features

- Pre-trained **DistilBERT** model for text classification.
- RESTful API built with **FastAPI**.
- **CORS** middleware enabled for browser-based access.
- Confidence-based responses for more granular feedback.

## Installation

To get started with the project, clone the repository and install the required dependencies.

### Clone the repository
```bash
git clone https://github.com/your-username/your-repo-name.git
cd your-repo-name


In [2]:
# Install the required libraries (uncomment if necessary)
# !pip install fastapi uvicorn transformers torch nest_asyncio

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from transformers import pipeline
import nest_asyncio
import uvicorn
from threading import Thread

# Apply nest_asyncio to fix event loop issues in Jupyter
nest_asyncio.apply()

# Load the pre-trained model for text classification (using DistilBERT)
classifier = pipeline('text-classification', model='distilbert-base-uncased')
# classifier = pipeline('text-classification', model='news-category-finetuned-model')


# Create a FastAPI app
app = FastAPI()

# Add CORS middleware to allow access from any browser
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins
    allow_methods=["*"],  # Allow all methods
    allow_headers=["*"]   # Allow all headers
)

# Define a route for text classification
@app.get("/classify/{text}")
async def classify(text: str):
    result = classifier(text)
    category = result[0]['label']
    confidence = result[0]['score']

    # Predefined example responses based on confidence and category
    if category == "POSITIVE" and confidence > 0.95:
        message = "The model is highly confident that the input text belongs to the category 'POSITIVE', with a confidence score of {:.2f}.".format(confidence)
    elif category == "NEGATIVE" and confidence > 0.95:
        message = "The model strongly believes that the input text belongs to the category 'NEGATIVE', with a high confidence score of {:.2f}.".format(confidence)
    elif category == "NEUTRAL":
        message = "The model predicts that the input text belongs to the category 'NEUTRAL' with a confidence score of {:.2f}.".format(confidence)
    elif category == "POSITIVE" and confidence < 0.60:
        message = "The model predicts that the input text might be 'POSITIVE', but the confidence score is low at {:.2f}.".format(confidence)
    elif category == "NEGATIVE" and confidence < 0.60:
        message = "The model predicts that the input text may belong to the category 'NEGATIVE', but the confidence is moderate at {:.2f}.".format(confidence)
    else:
        message = "The model predicts that the input text belongs to the category '{}' with a confidence score of {:.2f}.".format(category, confidence)
    
    # Returning response
    return {
        "category": category,
        "confidence": confidence,
        "message": message
    }

# Function to run FastAPI in the background
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start FastAPI in a separate thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


INFO:     Started server process [932]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53084 - "GET /classify/Hello HTTP/1.1" 200 OK
